# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
#be sure to include api_keys file with your api keys

In [10]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key


gmaps.configure(api_key=g_key)



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:

weather_df=pd.read_csv('../output_data/city_data.csv')
weather_df

,Unnamed: 0,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Min Temp,Wind Speed
0,0,534313,Lomovka,100,RU,1.644555e+09,93,53.9194,58.3672,4.12,4.12,4.63
1,1,2108857,Gizo Government Station,74,SB,1.644555e+09,80,-8.1030,156.8419,80.91,80.91,7.83
2,2,2270385,Camacha,40,PT,1.644555e+09,94,33.0833,-16.3333,61.56,60.22,8.05
3,3,2460954,Araouane,26,ML,1.644555e+09,26,18.9048,-3.5265,58.26,58.26,14.20
4,4,964432,Port Alfred,3,ZA,1.644555e+09,82,-33.5906,26.8910,68.85,68.85,4.59
...,...,...,...,...,...,...,...,...,...,...,...,...
588,588,2511150,Santa Lucía,0,ES,1.644555e+09,45,27.9117,-15.5407,61.77,57.94,4.61
589,589,3356832,Henties Bay,83,NaN,1.644555e+09,89,-22.1160,14.2845,71.85,71.85,8.28
590,590,3526934,Hueyapan,11,MX,1.644555e+09,50,18.8667,-98.6667,56.08,56.08,2.06
591,591,2088122,Port Moresby,20,PG,1.644555e+09,55,-9.4431,147.1797,89.02,89.02,17.27


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
fig=gmaps.figure()
heat_layer= gmaps.heatmap_layer(weather_df[['Lat','Lng']],weights=weather_df['Humidity'])

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
ideal_df=weather_df[weather_df['Min Temp']>70]
ideal_df=ideal_df[ideal_df['Max Temp']<80]
ideal_df=ideal_df[ideal_df['Wind Speed']<10]
ideal_df=ideal_df[ideal_df['Cloudiness']==0]


ideal_df

,Unnamed: 0,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Min Temp,Wind Speed
44,44,1254046,Tura,0,IN,1.644555e+09,42,25.5198,90.2201,70.57,70.57,5.21
62,62,3447597,Sidrolândia,0,BR,1.644555e+09,79,-20.9319,-54.9614,75.99,75.99,5.59
212,212,2455216,Kita,0,ML,1.644555e+09,10,13.0349,-9.4895,70.70,70.70,8.41
373,373,4026082,Lázaro Cárdenas,0,MX,1.644555e+09,72,17.9583,-102.2000,70.29,70.29,4.81
477,477,1174451,Keti Bandar,0,PK,1.644555e+09,40,24.1447,67.4497,72.55,72.55,4.45
488,488,3932145,Pisco,0,PE,1.644555e+09,73,-13.7000,-76.2167,71.65,71.02,6.91
583,583,3520994,Puerto Escondido,0,MX,1.644555e+09,83,15.8500,-97.0667,77.83,77.83,0.00


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
Hotel_Names=[]
for i in range(0,len(ideal_df['City'])):
    


    # geocoordinates
    target_coordinates =f" {ideal_df.iloc[i]['Lat']},{ideal_df.iloc[i]['Lng']}"
    
    target_radius = 5000
    target_type = "hotel"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)


    Hotel_Names.append(response.json()['results'][0]['name'])

In [51]:
ideal_df['Hotel Name']=Hotel_Names
ideal_df

,Unnamed: 0,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Min Temp,Wind Speed,Hotel_Names,Hotel Name
44,44,1254046,Tura,0,IN,1.644555e+09,42,25.5198,90.2201,70.57,70.57,5.21,Tura,Tura
62,62,3447597,Sidrolândia,0,BR,1.644555e+09,79,-20.9319,-54.9614,75.99,75.99,5.59,Sidrolândia,Sidrolândia
212,212,2455216,Kita,0,ML,1.644555e+09,10,13.0349,-9.4895,70.70,70.70,8.41,Kita,Kita
373,373,4026082,Lázaro Cárdenas,0,MX,1.644555e+09,72,17.9583,-102.2000,70.29,70.29,4.81,Lázaro Cárdenas,Lázaro Cárdenas
477,477,1174451,Keti Bandar,0,PK,1.644555e+09,40,24.1447,67.4497,72.55,72.55,4.45,Keti Bandar,Keti Bandar
488,488,3932145,Pisco,0,PE,1.644555e+09,73,-13.7000,-76.2167,71.65,71.02,6.91,Pisco,Pisco
583,583,3520994,Puerto Escondido,0,MX,1.644555e+09,83,15.8500,-97.0667,77.83,77.83,0.00,Puerto Escondido,Puerto Escondido


In [52]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_df.iterrows()]
locations = ideal_df[["Lat", "Lng"]]

In [53]:
# Add marker layer ontop of heat map
fig=gmaps.figure()
heat_layer= gmaps.heatmap_layer(ideal_df[['Lat','Lng']],weights=ideal_df['Humidity'])

fig.add_layer(heat_layer)

fig

# Display figure


Figure(layout=FigureLayout(height='420px'))